In [2]:
api_key = "abRJTjZ5f940NqDm"
api_secret = "iZUizAftYAExBKIFktworjDksAWajsHU"

In [5]:
from coinbase.wallet.client import Client

client = Client(api_key, api_secret, api_version='202-01-01')

currency_code = 'USD'  # can also use EUR, CAD, etc.

# Make the request
price = client.get_spot_price(currency=currency_code)

print('Current link price in %s: %s' % (currency_code, price.amount))

Current link price in USD: 17052.66
